## Summary of the results

I have run 25 experiments. Normally 30 epochs per set up which took around 30h on a p2.xlarge instance in AWS.

In [10]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re

from pathlib import Path

In [5]:
DATA_PATH = Path("../datasets/Amazon/models/")
fname = "results_df.p"

In [7]:
df_results = pd.read_pickle(DATA_PATH/fname)

These are all experiments I run, order by name

In [11]:
df_results.sort_values('modelname')

,modelname,iter_loss,best_hr,best_ndcg,best_iter,train_time
1,GMF_bs_1024_lr_001_n_emb_8_lrs_wolrs.pt,0.062381,0.684681,0.457842,26,77.529552
6,GMF_bs_512_lr_0001_n_emb_64_lrnr_adam_lrs_wolrs.pt,0.022108,0.429437,0.261455,30,178.078980
2,GMF_bs_512_lr_001_n_emb_16_lrs_wolrs.pt,0.047171,0.643982,0.426472,26,107.935192
11,GMF_bs_512_lr_001_n_emb_64_lrnr_adam_lrs_wolrs.pt,0.029333,0.541425,0.356743,29,181.950120
4,GMF_bs_512_lr_001_n_emb_8_lrnr_RMSprop_lrs_wlrs.pt,0.137572,0.535116,0.367750,17,104.829468
3,GMF_bs_512_lr_001_n_emb_8_lrnr_SGD_lrs_wlrs.pt,0.160001,0.103824,0.047371,13,94.832950
5,GMF_bs_512_lr_001_n_emb_8_lrnr_adam_lrs_wlrs.pt,0.066104,0.687512,0.452048,29,94.790884
0,GMF_bs_512_lr_001_n_emb_8_lrs_wolrs.pt,0.065170,0.700403,0.464460,29,94.523185
19,GMF_bs_512_lr_003_n_emb_16_lrnr_adam_lrs_wolrs.pt,0.106611,0.914795,0.631789,29,103.750941
20,GMF_bs_512_lr_003_n_emb_32_lrnr_adam_lrs_wolrs.pt,0.176158,0.999435,0.629910,29,122.771667


The way one should read the `modelname` is, for example:
    
    MLP_bs_512_reg_00_lr_001_n_emb_128_ll_64_dp_wodp_lrnr_adam_lrs_wolrs.pt
    
This is the MLP model, with batch size of 512, no regularization, learning rate 0.01, using embeddings of dimension 128, the last layer has 64 units, no dropout, Adam optimizer and no learning rate scheduler. 

Let's start by having a look to the GMF results:

### GMF

In [ ]:
df_results[df_results.modelname.str.contains('GMF')].sort_values('')

In [32]:
(df_results[df_results.modelname.str.contains('GMF')]
     .sort_values('best_hr', ascending=False)
     .reset_index(drop=True)
)

,modelname,iter_loss,best_hr,best_ndcg,best_iter,train_time
0,GMF_bs_512_lr_003_n_emb_64_lrnr_adam_lrs_wolrs.pt,0.179048,0.999605,0.698754,30,178.450981
1,GMF_bs_512_lr_003_n_emb_32_lrnr_adam_lrs_wolrs.pt,0.176158,0.999435,0.629910,29,122.771667
2,GMF_bs_512_lr_003_n_emb_64_lrnr_adam_lrs_wlrs.pt,0.141146,0.993296,0.665000,30,187.978307
3,GMF_bs_512_lr_003_n_emb_16_lrnr_adam_lrs_wolrs.pt,0.106611,0.914795,0.631789,29,103.750941
4,GMF_bs_512_lr_003_n_emb_8_lrnr_adam_lrs_wolrs.pt,0.102676,0.790247,0.584697,30,92.316454
5,GMF_bs_512_lr_001_n_emb_8_lrs_wolrs.pt,0.065170,0.700403,0.464460,29,94.523185
6,GMF_bs_512_lr_001_n_emb_8_lrnr_adam_lrs_wlrs.pt,0.066104,0.687512,0.452048,29,94.790884
7,GMF_bs_1024_lr_001_n_emb_8_lrs_wolrs.pt,0.062381,0.684681,0.457842,26,77.529552
8,GMF_bs_512_lr_001_n_emb_16_lrs_wolrs.pt,0.047171,0.643982,0.426472,26,107.935192
9,GMF_bs_512_lr_003_n_emb_32_lrnr_adam_lrs_wolrs_BCELoss.pt,0.329084,0.623072,0.394799,29,128.700660


There are a number of conclusion drawn from that table:

1. The more embeddings (n_emb) the better.

    * As we can see in the table, the best results are achieved when n_emb equals 64.


2. Learning rate scheduling does not seem to help much.

    * these days most DL problems benefit from using some form of learning rate scheduling. In particular, [Cyclic Learning Rates](https://arxiv.org/pdf/1506.01186.pdf) seem to lead to the best results. In this particular exercise, this does not seem to be the case.

        
3. This problem allows for high (0.03) learning rates, at least compared to what I am used to. 

    * The most significant jump in both HR@10 and NDCG@10 happens when incresing the learning rate from 0.01 to 0.03


4. When building recommendation algorithms one should be careful when selecting the measure of success.

    * This is perhaps the most important aspect of all. And deserves a bit more attent

In [20]:
modelname = df_gmf.modelname
modelname = [re.sub(".h5|.pt|.params|_reg_00|_loss_mse", "", n) for n in modelname]

In [23]:
modelname

['GMF_bs_512_lr_001_n_emb_8_lrs_wolrs',
 'GMF_bs_1024_lr_001_n_emb_8_lrs_wolrs',
 'GMF_bs_512_lr_001_n_emb_16_lrs_wolrs',
 'GMF_bs_512_lr_001_n_emb_8_lrnr_SGD_lrs_wlrs',
 'GMF_bs_512_lr_001_n_emb_8_lrnr_RMSprop_lrs_wlrs',
 'GMF_bs_512_lr_001_n_emb_8_lrnr_adam_lrs_wlrs',
 'GMF_bs_512_lr_0001_n_emb_64_lrnr_adam_lrs_wolrs',
 'GMF_bs_512_lr_001_n_emb_64_lrnr_adam_lrs_wolrs',
 'GMF_bs_512_lr_003_n_emb_64_lrnr_adam_lrs_wlrs',
 'GMF_bs_512_lr_003_n_emb_64_lrnr_adam_lrs_wolrs',
 'GMF_bs_512_lr_003_n_emb_8_lrnr_adam_lrs_wolrs',
 'GMF_bs_512_lr_003_n_emb_16_lrnr_adam_lrs_wolrs',
 'GMF_bs_512_lr_003_n_emb_32_lrnr_adam_lrs_wolrs',
 'GMF_bs_512_lr_003_n_emb_32_lrnr_adam_lrs_wolrs_BCELoss']